In [1]:
import cirq
from random import choices
import binascii

Here is a basic Demonstration of QKD and the Quantum Protocol BB84, whilst showing the roles of Alice, Bob, and our "hacker" Eve. 
------------------------------------------

~Part 1~ Alice and Bob utilize BB84 in order to share a secret key for their channel of communication. 
---------------------------------------
-phase 1
1. Alice randomly Chooses Bits
2. Alice Randomly Chooses Bases
3. Alice creates qubits
4. Alice sends qubits to Bob

-phase 2

5. Bob randomly chooses bases
6. Bob measures the qubits
7. Bob creates a key

-phase 3

8. Alice and Bob Compare Bases
9. Alice and Bob Compare the First bits in their key

In [3]:
#initialization 
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, "X": cirq.H}

num_bits = 10
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

In [4]:
# 1.

alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

Alice's initial key:  [0, 1, 1, 1, 1, 0, 0, 1, 0, 1]


In [5]:
# 2.
alice_bases = choices(["Z", "X"], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)


Alice's randomly chosen bases:  ['X', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z']


In [6]:
# 3.
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

In [8]:
# 4.

'''
This step is where Alice would send Bob the qubits.

'''

'\nThis step is where Alice would send Bob the qubits.\n\n'

In [7]:
# 5. 
bob_bases = choices(["Z", "X"], k = num_bits)


bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))


In [9]:
# 6.
bob_circuit.append(cirq.measure(qubits, key = "Bob's key"))

print(bob_circuit)

q0: ───H───M('Bob's key')───
           │
q1: ───H───M────────────────
           │
q2: ───H───M────────────────
           │
q3: ───I───M────────────────
           │
q4: ───H───M────────────────
           │
q5: ───I───M────────────────
           │
q6: ───H───M────────────────
           │
q7: ───H───M────────────────
           │
q8: ───I───M────────────────
           │
q9: ───H───M────────────────


In [10]:
# 7. 
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements["Bob's key"][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [0 1 1 0 1 1 1 0 0 1]


In [11]:
# 8. 
final_alice_key = []
final_bob_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)


Alice's key:  [0, 1, 1, 1]
Bob's key:  [0, 1, 1, 1]


In [12]:
# 9. 
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



We can use our keys!
Alice Key:  [1, 1]
Bob Key:  [1, 1]


~Part 2~ BB84 - Alice, Bob and Eve's Measurement Attack
----------------------------------
 -Phase 1

 1. Alice randomly chooses bits
 2. Alice randomly chooses bases
 3. Alice creates qubits
 4. Alice sends the qubits to Bob

 -Eve Intercepts

 5. Eve randomly Chooses Bases
 6. Eve Measures the qubits
 7. Eve Creates a Key
 8. Eve repeats step 3 to fool Bob

 -Phase 2

 9. Bob randomly chooses Bases
 10. Bob measures the qubits
 11. Bob creates a key

 -Phase 3

 12. Alice and Bob compare bases
 13. Alice and Bob Compare the First Bits in Their Key



In [13]:
# Initialization
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}

num_bits = 10
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

In [14]:
# 1
alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

Alice's initial key:  [1, 0, 1, 1, 1, 0, 1, 0, 1, 0]


In [15]:
# 2
alice_bases = choices(['Z', 'X'], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)


Alice's randomly chosen bases:  ['X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z']


In [16]:
# 3
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

print(alice_circuit)

q0: ───X───H───

q1: ───I───I───

q2: ───X───I───

q3: ───X───I───

q4: ───X───I───

q5: ───I───H───

q6: ───X───H───

q7: ───I───H───

q8: ───X───H───

q9: ───I───I───


In [ ]:
# 4
'''
Alice sends qubits to Bob

'''

In [17]:
# 5 (Eve Intercepts)
#For this attack, Eve does not try to guess bases, so no code is required.

In [20]:
# 6
eve_circuit = cirq.Circuit()
eve_circuit.append(cirq.measure(qubits, key = "Eve's key"))

In [21]:
# 7
eve_intercept_circuit = alice_circuit + eve_circuit

sim = cirq.Simulator()
results = sim.run(eve_intercept_circuit)
eve_key = results.measurements["Eve's key"][0]

print('\nEve\'s initial key: ', eve_key)


Eve's initial key:  [0 0 1 1 1 1 1 1 1 0]


In [22]:
# 8
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = eve_key[bit]
  encode_gate = encode_gates[encode_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))

print('\nAlice\'s Phase 1 circuit after Eve\'s interception:\n', alice_circuit)


Alice's Phase 1 circuit after Eve's interception:
 q0: ───I───

q1: ───I───

q2: ───X───

q3: ───X───

q4: ───X───

q5: ───X───

q6: ───X───

q7: ───X───

q8: ───X───

q9: ───I───


In [23]:
# 9 (Bob recieves)
bob_bases = choices(["Z", "X"], k= num_bits)


bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

In [24]:
# 10
bob_circuit.append(cirq.measure(qubits, key= "Bob's key"))

print(bob_circuit)

q0: ───I───M('Bob's key')───
           │
q1: ───H───M────────────────
           │
q2: ───I───M────────────────
           │
q3: ───I───M────────────────
           │
q4: ───H───M────────────────
           │
q5: ───H───M────────────────
           │
q6: ───H───M────────────────
           │
q7: ───H───M────────────────
           │
q8: ───H───M────────────────
           │
q9: ───I───M────────────────


In [25]:
# 11
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements["Bob's key"][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [0 1 1 1 0 0 1 1 0 0]


In [26]:
# 12
final_alice_key = []
final_bob_key = []
final_eve_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])
    final_eve_key.append(eve_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)


Alice's key:  [1, 1, 0, 1, 0, 1, 0]
Bob's key:  [1, 1, 0, 1, 1, 0, 0]
Eve's key:  [1, 1, 1, 1, 1, 1, 0]


In [27]:
# 13
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



We can use our keys!
Alice Key:  [1, 0, 1, 0]
Bob Key:  [1, 1, 0, 0]
Eve Key:  [1, 1, 1, 0]


In [ ]:
'''
You will notice that the code said that we can use our keys, however,
we only compared the first 3 digits of the key, and thus the program believes these 
keys are valid, but after removing the qubits that were shared
the final key between Alice and Bob are not identical therefore
proving that Eve was listening. 

This is why when doing QKD: BB84, it is good measure to use a lot of qubits as opposed
to 7-10 qubits. 
'''

~Part 3~ Alice, Bob, and Eve's Intercept and Resend Attack
---------------
-Phase 1

1. Alice Randomly Chooses Bits
2. Alice Randomly Chooses Bases
3. Alice Creates Qubits
4. Alice sends qubits to Bob

-Eve Intercepts

5. Eve Randomly chooses bases
6. Eve measures the qubits
7. Eve creates a key
8. Eve repeats step 3 to fool Bob then sends to Bob

-Phase 2 

9. Bob Randomly Chooses Bases
10. Bob measures the qubits
11. Bob creates a key

-Phase 3

12. Alice and Bob compare bases
13. Alice and Bob compare the first bits in their key

In [28]:
#initialization 
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {"Z": cirq.I, "X": cirq.H}

num_bits = 100
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

In [29]:
# 1
alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

Alice's initial key:  [1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0]


In [30]:
# 2
alice_bases = choices(["Z", "X"], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)


Alice's randomly chosen bases:  ['Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z']


In [31]:
# 3
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

In [ ]:
# 4
'''
Alice sends qubits to Bob

'''

In [32]:
# 5 (Eve Intercepts)
eve_bases = choices(["Z", "X"], k = num_bits)
print('Eve\'s randomly chosen bases: ', eve_bases)

eve_circuit = cirq.Circuit()

for bit in range(num_bits):
  basis_value = eve_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  eve_circuit.append(basis_gate(qubit))

Eve's randomly chosen bases:  ['X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z']


In [33]:
# 6 
eve_circuit.append(cirq.measure(qubits, key = 'eve key'))

In [34]:
# 7
eve_intercept_circuit = alice_circuit + eve_circuit

sim = cirq.Simulator()
results = sim.run(eve_intercept_circuit)
eve_key = results.measurements['eve key'][0]

print('\nEve\'s initial key: ', eve_key)


Eve's initial key:  [0 0 1 1 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 1 1 1 1 1
 0 1 0 1 0 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0
 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0]


In [35]:
# 8
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = eve_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = eve_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

print('\nAlice\'s Phase 1 circuit after Eve\'s interception:\n', alice_circuit)


Alice's Phase 1 circuit after Eve's interception:
 q0: ────I───H───

q1: ────I───I───

q2: ────X───I───

q3: ────X───I───

q4: ────X───I───

q5: ────I───I───

q6: ────X───I───

q7: ────X───I───

q8: ────I───H───

q9: ────I───I───

q10: ───X───I───

q11: ───I───I───

q12: ───I───I───

q13: ───X───I───

q14: ───I───H───

q15: ───X───I───

q16: ───I───H───

q17: ───I───I───

q18: ───I───H───

q19: ───I───H───

q20: ───X───H───

q21: ───X───H───

q22: ───X───I───

q23: ───I───I───

q24: ───X───I───

q25: ───X───H───

q26: ───X───I───

q27: ───I───H───

q28: ───I───H───

q29: ───I───I───

q30: ───I───I───

q31: ───I───I───

q32: ───X───I───

q33: ───X───H───

q34: ───X───H───

q35: ───X───I───

q36: ───X───I───

q37: ───I───I───

q38: ───X───H───

q39: ───I───I───

q40: ───X───I───

q41: ───I───H───

q42: ───I───H───

q43: ───X───I───

q44: ───I───H───

q45: ───I───I───

q46: ───X───I───

q47: ───X───I───

q48: ───X───H───

q49: ───I───H───

q50: ───I───I───

q51: ───I───H───

q52: ───X───

In [36]:
# 9 (Bob recieves)
bob_bases = choices(["Z", "X"], k = num_bits)


bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

In [37]:
# 10
bob_circuit.append(cirq.measure(qubits, key = "bob key"))

print(bob_circuit)

q0: ────H───M('bob key')───
            │
q1: ────H───M──────────────
            │
q2: ────H───M──────────────
            │
q3: ────I───M──────────────
            │
q4: ────I───M──────────────
            │
q5: ────I───M──────────────
            │
q6: ────H───M──────────────
            │
q7: ────I───M──────────────
            │
q8: ────H───M──────────────
            │
q9: ────H───M──────────────
            │
q10: ───H───M──────────────
            │
q11: ───I───M──────────────
            │
q12: ───H───M──────────────
            │
q13: ───I───M──────────────
            │
q14: ───I───M──────────────
            │
q15: ───H───M──────────────
            │
q16: ───H───M──────────────
            │
q17: ───I───M──────────────
            │
q18: ───I───M──────────────
            │
q19: ───H───M──────────────
            │
q20: ───I───M──────────────
            │
q21: ───I───M──────────────
            │
q22: ───H───M──────────────
            │
q23: ───I───M──────────────
      

In [38]:
# 11
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 1 1
 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 1 0
 1 1 0 0 1 0 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 0]


In [39]:
# 12
final_alice_key = []
final_bob_key = []
final_eve_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])
    final_eve_key.append(eve_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)


Alice's key:  [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0]
Bob's key:  [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
Eve's key:  [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [40]:
# 13
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



Eve was listening, we need to use a different channel!


~Part 4~ Alice, Bob, and Eve's Entanglement Attack
------------------------
-Phase 1

1. Alice randomly chooses Bits
2. Alice randomly chooses Bases
3. Alice creates Qubits
4. Alice sends qubits to Bob

-Eve intercepts

5. Eve randomly entagles her qubits

-Phase 2

6. Bob randomly Chooses Bases
7. Bob measures the qubits
8. Bob creates a key

-Phase 3

9. Alice and Bob compare Bases
10. Alice and Bob compare the first bits in their key


In [53]:
#Initialization
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {"Z": cirq.I, "X": cirq.H}

num_bits = 30
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

In [54]:
# 1
alice_key = choices([0, 1], k = num_bits)

print('Alice\'s initial key: ', alice_key)

Alice's initial key:  [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1]


In [55]:
# 2
alice_bases = choices(["Z", "X"], k = num_bits)

print('\nAlice\'s randomly chosen bases: ', alice_bases)


Alice's randomly chosen bases:  ['Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z']


In [56]:
# 3
alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_value = alice_key[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

In [ ]:
# 4
'''
Sends qubits to Bob

'''

In [57]:
# 5 (Eve Intercepts)
eve_qubits = cirq.NamedQubit.range(num_bits, prefix = 'e')

for bit in range(num_bits):

  qubit = qubits[bit]
  eve_qubit = eve_qubits[bit]

  alice_circuit.append(cirq.CNOT(qubit, eve_qubit))

In [58]:
print(alice_circuit)

                ┌──────────────────────────────┐
e0: ─────────────X─────────────────────────────────
                 │
e1: ─────────────┼X────────────────────────────────
                 ││
e2: ─────────────┼┼X───────────────────────────────
                 │││
e3: ─────────────┼┼┼X──────────────────────────────
                 ││││
e4: ─────────────┼┼┼┼X─────────────────────────────
                 │││││
e5: ─────────────┼┼┼┼┼X────────────────────────────
                 ││││││
e6: ─────────────┼┼┼┼┼┼X───────────────────────────
                 │││││││
e7: ─────────────┼┼┼┼┼┼┼X──────────────────────────
                 ││││││││
e8: ─────────────┼┼┼┼┼┼┼┼X─────────────────────────
                 │││││││││
e9: ─────────────┼┼┼┼┼┼┼┼┼X────────────────────────
                 ││││││││││
e10: ────────────┼┼┼┼┼┼┼┼┼┼X───────────────────────
                 │││││││││││
e11: ────────────┼┼┼┼┼┼┼┼┼┼┼X──────────────────────
                 ││││││││││││
e12: ────────────┼┼┼┼┼┼┼┼┼┼┼┼X───

In [59]:
# 6 (Bob Recieves)
bob_bases = choices(["Z", "X"], k = num_bits)
print('Bob\'s randomly chosen bases: ', bob_bases)

bob_circuit = cirq.Circuit()

for bit in range(num_bits):

  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

Bob's randomly chosen bases:  ['X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z']


In [60]:
# 7
bob_circuit.append(cirq.measure(qubits, key = "bob key"))

print(bob_circuit)

q0: ────H───M('bob key')───
            │
q1: ────H───M──────────────
            │
q2: ────H───M──────────────
            │
q3: ────I───M──────────────
            │
q4: ────I───M──────────────
            │
q5: ────H───M──────────────
            │
q6: ────I───M──────────────
            │
q7: ────I───M──────────────
            │
q8: ────I───M──────────────
            │
q9: ────I───M──────────────
            │
q10: ───I───M──────────────
            │
q11: ───H───M──────────────
            │
q12: ───H───M──────────────
            │
q13: ───H───M──────────────
            │
q14: ───H───M──────────────
            │
q15: ───I───M──────────────
            │
q16: ───I───M──────────────
            │
q17: ───H───M──────────────
            │
q18: ───I───M──────────────
            │
q19: ───I───M──────────────
            │
q20: ───I───M──────────────
            │
q21: ───H───M──────────────
            │
q22: ───I───M──────────────
            │
q23: ───I───M──────────────
      

In [61]:
# 8
bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

print('\nBob\'s initial key: ', bob_key)


Bob's initial key:  [1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1 0 0 0 1 1]


In [62]:
# 9
final_alice_key = []
final_bob_key = []
final_eve_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_bases[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])
    final_eve_key.append(eve_key[bit])

In [63]:
basis_value = bob_bases[bit]
basis_gate = basis_gates[basis_value]

qubit = eve_qubits[bit]
bb84_circuit.append(basis_gate(qubit))
bb84_circuit.append(cirq.measure(qubit))

sim = cirq.Simulator()
results = sim.run(bb84_circuit)

final_eve_key.append(results.measurements['e' + str(bit)][0][0])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)
print('Eve\'s key: ', final_eve_key)


Alice's key:  [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1]
Bob's key:  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1]
Eve's key:  [0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1]


In [64]:
# 10
num_bits_to_compare = int(len(final_alice_key) * .5)
if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
  final_alice_key = final_alice_key[num_bits_to_compare:]
  final_bob_key = final_bob_key[num_bits_to_compare:]
  final_eve_key = final_eve_key[num_bits_to_compare:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)
  print('Eve Key: ', final_eve_key)

else:
  print('\n\nEve was listening, we need to use a different channel!')



Eve was listening, we need to use a different channel!
